# Backlog APIでwikiページ情報情報を取得するサンプル

## 設定

In [ ]:
USE_PROJECT_KEYS: list[str] = [
    # 'BWKM',
    'BSEV',
]

In [ ]:
SKIP_WIKI_PAGE_NAME: list[str] = [
    '不使用',
]

## BacklogのAPIキーを環境変数から取得する

In [ ]:
import os
BACKLOG_API_KEY = os.environ['BACKLOG_API_KEY']

In [ ]:
BACKLOG_BASE_URL = 'https://bitstar.backlog.jp'

## BacklogのAPIでユーザー情報を取得する

In [ ]:
API_URL = '/api/v2/projects'

In [ ]:
url = '{base_url}{api_url}?apiKey={api_key}'.format(base_url = BACKLOG_BASE_URL, api_url = API_URL, api_key = BACKLOG_API_KEY,)

In [ ]:
import requests

In [ ]:
response = requests.get(url)

In [ ]:
response

In [ ]:
# response.text

In [ ]:
import json
data = json.loads(response.text)

## JSONをパースする

In [ ]:
import json
projects = json.loads(response.text)

In [ ]:
# projects

In [ ]:
project_name_dict: dict[str] = {}
project_key_dict: dict[str] = {}
for project in projects:
    if project['projectKey'] not in USE_PROJECT_KEYS:
        continue
    project_name_dict[str(project['id'])] = project['name']
    project_key_dict[str(project['id'])] = project['projectKey']

In [ ]:
project_name_dict

In [ ]:
keys = ['projectKey', 'name']
projects = [{key: row[key] for key in keys} for row in data]
projects

## BacklogのAPIでwikiページ一覧を取得する

In [ ]:
API_URL = '/api/v2/wikis'

In [ ]:
wiki_pages: list[dict] = []
for project in projects:
    if project['projectKey'] not in USE_PROJECT_KEYS:
        continue
    
    url = '{base_url}{api_url}?apiKey={api_key}&projectIdOrKey={project_key}'.format(base_url = BACKLOG_BASE_URL, api_url = API_URL, api_key = BACKLOG_API_KEY, project_key = project['projectKey'],)
    response = requests.get(url)

    data = json.loads(response.text)

    for row in data:
        skip = False
        for name in SKIP_WIKI_PAGE_NAME:
            if name in row['name']:
                skip = True

        if skip:
            continue

        row['projectKey'] = project_key_dict[str(row['projectId'])]
        row['projectName'] = project_name_dict[str(row['projectId'])]
        wiki_pages.append(row)
        # break
    # break

In [ ]:
wiki_pages[0]

In [ ]:
len(wiki_pages)

## Wikiページダウンロードフォルダの内容を削除する

In [ ]:
import os
import shutil
import glob

In [ ]:
files = glob.glob('./backlog-wikis/*')
for file in files:
    if os.path.isdir(file):
        shutil.rmtree(file)

## BacklogのAPIでwikiページ情報を取得する

In [ ]:
API_URL = '/api/v2/wikis/{wiki_id}'

In [ ]:
import time
import os
import shutil

for i, wiki_page in enumerate(wiki_pages):
    print(i, project_name_dict[str(wiki_page['projectId'])], wiki_page['id'], wiki_page['name'],)
    url = \
        '{base_url}{api_url}?apiKey={api_key}'.format(
            base_url = BACKLOG_BASE_URL,
            api_url = API_URL,
            api_key = BACKLOG_API_KEY,
        )
    url = url.format(wiki_id = wiki_page['id'],)
    response = requests.get(url)

    data = json.loads(response.text)

    content_dir_path: str = \
        './backlog-wikis/{project_key}_{project_name}/{wiki_id}_{name}/'.format(
            project_key = wiki_page['projectKey'],
            project_name = wiki_page['projectName'],
            wiki_id = wiki_page['id'],
            name = wiki_page['name'].replace('/', '／'),
        )

    if os.path.isdir(content_dir_path):
        shutil.rmtree(content_dir_path)

    os.makedirs(content_dir_path)

    content_file_path: str = '{dir}content.md'.format(dir = content_dir_path,)

    with open(content_file_path, 'w') as f:
        f.write(data['content'])

    time.sleep(3)